In [ ]:
# Written to read and explore the initial data
# associating drug labels to adverse events
# written 12-13-18 JLW, adapted 8-28-19

import pickle,os,csv
from collections import defaultdict

rscs_dir = '../rscs/'
data_dir = '../data/'

db2nf = os.path.join(rscs_dir,'drugbankid_to_name.pkl')
db2n = pickle.load(open(db2nf,'rb'))
n2db = dict([(nm.lower(),db) for (db,nm) in db2n.items()])

In [ ]:
# methods for phenotype matching
dme_syns = pickle.load(open(os.path.join(data_dir,'synonyms.pkl'),'rb'))
def strip_string(s):
        return s.replace(',','').replace('(','').replace(')','')

def exact_overlap(t,p): # check if all tox words are contained in phenotype
        t_words = [x.lower() for x in t.split(' ')]
        p_words = [x.lower() for x in strip_string(p).split(' ')]
        t_len = float(len(t_words))
        if len(set(t_words).intersection(set(p_words))) == t_len:
                return True
        else:
                return False

def check_synonyms(t,p):
        match = False
        if dme_syns[t] != []:
                syns = dme_syns[t]
                syns = [x for x in syns if x!= '']
                for s in syns:
                        if exact_overlap(s,p):
                                match = True
        return match

In [ ]:
# load drug-DME data from drug labels
f = os.path.join(data_dir,'Drugs_labeled_for_AEs.txt')
dR = csv.DictReader(open(f,'r'),delimiter='\t')

# gather starting data
unique_drugs = set()
drugs_by_dme = defaultdict(list)
dmes_by_drugs = defaultdict(list)
for r in dR:
        for (dme,drug) in r.items():
                if drug != '':
                        drugs_by_dme[dme].append(drug.lower())
                        unique_drugs.add(drug.lower())
                        dmes_by_drugs[drug.lower()].append(dme)
print('extracted drug-dme relationships')

# create set of false positives
fpdmes_by_drugs = defaultdict(list)
unique_dmes = [k for k in drugs_by_dme.keys()]
for (drug,dme_list) in dmes_by_drugs.items():
        fp_dmes = list(set(unique_dmes).difference(set(dme_list))) # get list of dmes NOT on the drug label
        fpdmes_by_drugs[drug.lower()] = fp_dmes

# map to DrugBank IDs
un_in_DB = [n2db[n.lower()] for n in unique_drugs if n.lower() in n2db] #1136 drugs

In [ ]:
# check how many are in drug bank and modeled
db_with_targf = os.path.join(rscs_dir,'drugBank_with_at_least_one_intom_targ.pkl')
db_with_targ = pickle.load(open(db_with_targf,'rb'))
un_with_targ = [n for n in un_in_DB if n in db_with_targ] #997 drugs
un_with_targ_dname = [db2n[d].lower() for d in un_with_targ] #997 drugs
num_drug_analyzed = len(un_with_targ)
un_w_t_in_dme = [d for d in un_with_targ if db2n[d].lower() in dmes_by_drugs] # drug with targets and dmes
print('loaded drug bank mappings')

# dictionary to look up drug network results
asfdir = os.path.join(data_dir,'all_drugbank_network_association_files/')
asf = [f for f in os.listdir(asfdir)]
asf_dic = dict([(f.split('_')[0],f) for f in asf if 'merged_neighborhood__assoc_table_.txt' in f])
print('loaded association files')

# other data needed for analysis
# expected p-values for each dme, and number of drug targets
back_phmeds_f = os.path.join(rscs_dir,'back_phmeds.pkl')
back_phmeds = pickle.load(open(back_phmeds_f,'rb')) # to find expected p-value for phenotype
dintf = os.path.join(rscs_dir,'drug_intome_targets.pkl')
dint = pickle.load(open(dintf,'rb')) # to find number of drug targets

In [ ]:
# run analysis, store data for calculating specificity
drugs_matched_to_tox = defaultdict(set)
drugs_fp_to_tox = defaultdict(set)
of1 = open('drugs_to_dmes_true_positive.txt','w')
hdr = ['Drug Name','DrugBankID','Label DME','PathFX Phenotype','PathFX Phen. CUI','Corrected P-value','Network genes assoc to phen','\n']
of1.write('\t'.join(hdr))
of2 = open('drugs_to_dmes_false_positives.txt','w')
hdr = ['Drug Name','DrugBankID','False Positive DME','PathFX Phenotype','PathFX Phen. CUI','Corrected P-value','Network genes assoc to phen','\n']
of2.write('\t'.join(hdr))
for dbid in un_w_t_in_dme:
        print('analyzing drug: '+dbid)
        asf = os.path.join(asfdir,asf_dic[dbid])
        targs = dint[dbid]
        nt = len(targs)
        dname = db2n[dbid].lower()
        drug_dmes = dmes_by_drugs[dname]
        fp_dmes = fpdmes_by_drugs[dname]
        dR = csv.DictReader(open(asf,'rU'),delimiter='\t')
        for row in dR: # get each phenotype for that drug
                [rank,ph,cui,asin,asii,BH,genes] = [row['rank'],row['phenotype'],row['cui'],row['assoc in neigh'],row['assoc in intom'],float(row['Benjamini-Hochberg']),row['genes']]
                if cui in back_phmeds[nt]:
                        exp_BH = back_phmeds[nt][cui]
                else:
                        exp_BH = 1
                if BH < exp_BH: # only look for matches with the significant phenotypes
                        for tox in drug_dmes:
                                if exact_overlap(tox,ph) or check_synonyms(tox,ph):
                                        drugs_matched_to_tox[tox].add(dname)
                                        out_data = [dname,dbid,tox,ph,cui,str(exp_BH),genes,'\n']
                                        of1.write('\t'.join(out_data))

                        for tox in fp_dmes:
                                if exact_overlap(tox,ph) or check_synonyms(tox,ph):
                                        drugs_fp_to_tox[tox].add(dname)
                                        out_data = [dname,dbid,tox,ph,cui,str(exp_BH),genes,'\n']
                                        of2.write('\t'.join(out_data))
of1.close()
of2.close()

In [ ]:
# calculate specificity and sensitivity
of3 = open('dme_label_specificity_sensitivity.txt','w')
hdr = ['DME','True Positive Count','True Positives','False Negative Count','False Negatives','False Positive Count','False Positives','True Negative Count','True Negatives','\n']
of3.write('\t'.join(hdr))
for dme in drugs_by_dme:
        print('TP,FP calculation for: '+dme)
        all_positives = drugs_by_dme[dme]
        positives = set(all_positives).intersection(set(un_with_targ_dname)) # remove positives that aren't in the 997 drugs that are eligible for analysis
        print('num positives: '+str(len(positives)))
        true_positives = drugs_matched_to_tox[dme]
        print('num TP: '+str(len(true_positives)))
        false_negatives = list(set(positives).difference(true_positives)) # drugs with dmes not discovered by PathFX
        print('num FN: '+str(len(false_negatives)))

        negatives = list(set(un_with_targ_dname).difference(set(positives))) # drugs with targets but dme not on label
        print('num negatives: '+str(len(negatives)))
        false_positives = drugs_fp_to_tox[dme]
        print('num FP: '+str(len(false_positives)))
        true_negatives = list(set(negatives).difference(false_positives))
        print('num TN: '+str(len(true_negatives)))

        tpc = str(len(true_positives))
        tps = ','.join(list(true_positives))
        fnc = len(false_negatives)
        fns = ','.join(false_negatives)
        fpc = len(false_positives)
        fps = ','.join(list(false_positives))
        tnc = len(true_negatives)
        tns = ','.join(true_negatives)
        outdata = [dme,str(tpc),tps,str(fnc),fns,str(fpc),fps,str(tnc),tns,'\n']
        of3.write('\t'.join(outdata))
of3.close()